# Amazon SageMaker Autopilot을 활용한 다이렉트 마케팅
---

---

## 목차

1. [소개](#Introduction)
1. [사전 요구 사항](#Prerequisites)
1. [데이터셋 다운로드](#Downloading)
1. [Amazon S3에 데이터셋 업로드](#Uploading)
1. [SageMaker Autopilot 작업 설정](#Settingup)
1. [SageMaker Autopilot 작업 시작](#Launching)
1. [SageMaker Autopilot 작업 진행 상황 추적](#Tracking)
1. [결과](#Results)
1. [정리](#Cleanup)

## 소개

Amazon SageMaker Autopilot은 테이블 형식 데이터셋을 위한 자동화된 머신 러닝(일반적으로 AutoML이라고 함) 솔루션입니다. SageMaker Autopilot을 다양한 방식으로 사용할 수 있습니다: 자동 조종 모드(이름의 유래)로 또는 사용자 지침과 함께, SageMaker Studio를 통해 코드 없이, 또는 AWS SDK를 사용하여. 이 노트북은 첫 번째 시작으로 AWS SDK를 사용하여 간단하게 머신 러닝 모델을 생성하고 배포하는 방법을 보여줍니다.

머신 러닝의 전형적인 입문 과제("Hello World"와 동등한)는 데이터셋을 사용하여 고객이 한 번 이상의 전화 통화 후에 은행의 정기 예금에 가입할지 여부를 예측하는 것입니다. 이 작업과 사용된 데이터셋에 대한 자세한 정보는 [Bank Marketing Data Set](https://archive.ics.uci.edu/ml/datasets/bank+marketing)을 참조하세요.

우편, 이메일, 전화 등을 통한 직접 마케팅은 고객을 확보하는 일반적인 전략입니다. 자원과 고객의 관심이 제한되어 있기 때문에, 목표는 특정 제안에 참여할 가능성이 높은 잠재 고객의 하위 집합만을 대상으로 하는 것입니다. 인구통계, 과거 상호작용 및 환경적 요소와 같은 쉽게 이용할 수 있는 정보를 기반으로 이러한 잠재 고객을 예측하는 것은 일반적인 머신 러닝 문제입니다. 이 작업이 여러분 조직의 마케팅 리드 우선순위 지정으로 쉽게 변환될 수 있다고 상상할 수 있습니다.

이 노트북은 여러 잠재적 옵션 또는 "후보"를 탐색하여 가장 정확한 ML 파이프라인을 얻기 위해 이 데이터셋에 Autopilot을 사용하는 방법을 보여줍니다. Autopilot이 생성한 각 후보는 두 단계로 구성됩니다. 첫 번째 단계는 데이터셋에 대한 자동화된 특성 엔지니어링을 수행하고, 두 번째 단계는 모델을 생성하기 위해 알고리즘을 훈련하고 튜닝합니다. 이 모델을 배포할 때는 유사한 단계를 따릅니다. 특성 엔지니어링 후 추론을 통해 리드가 추구할 가치가 있는지 여부를 결정합니다. 이 노트북에는 모델을 훈련시키는 방법과 일련의 리드에 대한 배치 예측을 수행하기 위해 모델을 배포하는 방법에 대한 지침이 포함되어 있습니다. 가능한 경우, Amazon SageMaker Python SDK(고수준 SDK)를 사용하여 Amazon SageMaker와의 상호 작용 방식을 단순화합니다.

다른 예제들은 다양한 방식으로 모델을 사용자 정의하는 방법을 보여줍니다. 예를 들어, 기기에 배포된 모델은 일반적으로 정확성뿐만 아니라 메모리 제약 조건을 충족해야 합니다. 다른 사용 사례에는 실시간 배포 요구 사항과 지연 시간 제약이 있습니다. 지금은 간단하게 유지하겠습니다.

## 사전 요구 사항

이 튜토리얼의 작업을 시작하기 전에 다음을 수행하세요:

- 훈련 및 모델 데이터에 사용할 Amazon Simple Storage Service(Amazon S3) 버킷과 접두사. 이는 Amazon SageMaker 훈련과 동일한 리전에 있어야 합니다. 아래 코드는 기본 버킷을 생성하거나, 이미 존재하는 경우 해당 버킷을 사용합니다.
- Autopilot이 데이터에 접근할 수 있도록 하는 IAM 역할. IAM 역할에 대한 자세한 정보는 Amazon SageMaker 문서를 참조하세요: https://docs.aws.amazon.com/sagemaker/latest/dg/security-iam.html

In [ ]:
# cell 01
import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'sagemaker/autopilot-dm'

role = get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)

## 데이터셋 다운로드<a name="Downloading"></a>
샘플 데이터 s3 버킷에서 [다이렉트 마케팅 데이터셋](!wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip)을 다운로드합니다.

\[Moro et al., 2014\] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

In [ ]:
# cell 02
local_data_path = './bank-additional/bank-additional-full.csv'


## Amazon S3에 데이터셋 업로드하기<a name="Uploading"></a>

Autopilot으로 데이터셋을 실행하기 전에, 먼저 데이터셋에 명백한 오류가 없는지 확인해야 합니다. Autopilot 프로세스는 오랜 시간이 걸릴 수 있으므로, 작업을 시작하기 전에 데이터셋을 검사하는 것이 일반적으로 좋은 방법입니다. 이 특정 데이터셋은 작기 때문에 노트북 인스턴스 자체에서 검사할 수 있습니다. 노트북 인스턴스 메모리에 맞지 않는 더 큰 데이터셋이 있는 경우, Apache Spark와 같은 빅 데이터 분석 도구를 사용하여 오프라인으로 데이터셋을 검사하세요. [Deequ](https://github.com/awslabs/deequ)는 Apache Spark 위에 구축된 라이브러리로, 대규모 데이터셋에 대한 검사를 수행하는 데 도움이 될 수 있습니다. Autopilot은 최대 5GB 크기의 데이터셋을 처리할 수 있습니다.

Pandas 데이터 프레임으로 데이터를 읽고 살펴보겠습니다.

In [ ]:
# cell 03
import pandas as pd

data = pd.read_csv(local_data_path)
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

타겟 컬럼 'y'를 예측하는 데 도움이 되는 20개의 특성이 있습니다.

Amazon SageMaker Autopilot은 데이터 전처리를 대신 처리해 줍니다. 결측값 처리, 범주형 특성을 수치형 특성으로 변환, 데이터 스케일링, 더 복잡한 데이터 유형 처리와 같은 일반적인 데이터 전처리 기법을 수행할 필요가 없습니다.

또한, 데이터셋을 학습용과 검증용으로 분할할 필요도 없습니다. Autopilot이 이 작업을 대신 처리해 줍니다. 그러나 테스트 세트를 분리하고 싶을 수도 있습니다. 다음 단계가 바로 그것인데, 모델 테스트 대신 배치 추론을 위해 사용하게 됩니다.

### 모델에서 배치 추론을 호출하기 위한 데이터 예약

데이터를 훈련 및 테스트 분할로 나눕니다. 훈련 분할은 SageMaker Autopilot에서 사용됩니다. 테스트 분할은 제안된 모델을 사용하여 추론을 수행하기 위해 예약됩니다.

In [ ]:
# cell 04
train_data = data.sample(frac=0.8,random_state=200)

test_data = data.drop(train_data.index)

test_data_no_target = test_data.drop(columns=['y'])

### Amazon S3에 데이터셋 업로드하기
Amazon SageMaker 훈련에서 사용할 수 있도록 파일을 .csv 형식으로 Amazon Simple Storage Service(Amazon S3)에 복사합니다.

In [ ]:
# cell 05
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print('Train data uploaded to: ' + train_data_s3_path)

test_file = 'test_data.csv';
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print('Test data uploaded to: ' + test_data_s3_path)

## SageMaker Autopilot 작업 설정하기<a name="Settingup"></a>

데이터셋을 Amazon S3에 업로드한 후, Autopilot을 호출하여 이 데이터셋에서 모델을 훈련하기 위한 최적의 ML 파이프라인을 찾을 수 있습니다.

Autopilot 작업을 호출하기 위해 필요한 입력 사항은 다음과 같습니다:
* 입력 데이터셋과 모든 출력 아티팩트를 위한 Amazon S3 위치
* 예측하고자 하는 데이터셋의 열 이름(이 경우 `y`)
* IAM 역할

현재 Autopilot은 CSV 형식의 테이블 형태 데이터셋만 지원합니다. 모든 파일에 헤더 행이 있거나, 알파벳/사전 순으로 정렬했을 때 데이터셋의 첫 번째 파일에 헤더 행이 있어야 합니다.

In [ ]:
# cell 06
auto_ml_job_input_data_config = [{
  'DataSource': {
    'S3DataSource': {
      'S3DataType': 'S3Prefix',
      'S3Uri': 's3://{}/{}/train'.format(bucket,prefix)
    }
  }
}]

output_data_config = {
  'S3OutputPath': 's3://{}/{}/output'.format(bucket,prefix)
}

auto_ml_problem_type_config={
  'TabularJobConfig': {
    'CompletionCriteria': {
      'MaxCandidates': 5
    },
    'ProblemType': 'BinaryClassification',
    'TargetAttributeName': 'y',
  },
}

auto_ml_job_objective = {
  "MetricName": "Accuracy"
}

데이터셋으로 해결하고자 하는 문제 유형을 지정할 수도 있습니다(`Regression, MulticlassClassification, BinaryClassification`). 확실하지 않은 경우, SageMaker Autopilot은 대상 열(예측하려는 열)의 통계를 기반으로 문제 유형을 추론합니다.

SageMaker Autopilot 작업의 실행 시간을 제한하는 옵션이 있습니다. 파이프라인 평가 또는 후보의 최대 수(하나의 파이프라인 평가는 후보 모델을 생성하기 때문에 `Candidate`라고 함)를 제공하거나 전체 Autopilot 작업에 할당된 총 시간을 제공할 수 있습니다. 기본 설정에서 이 작업은 약 4시간이 소요됩니다. Autopilot이 최적의 훈련 매개변수를 찾기 위해 사용하는 탐색 프로세스의 특성으로 인해 실행마다 소요 시간이 달라질 수 있습니다.

## SageMaker Autopilot 작업 시작하기<a name="Launching"></a>

이제 `create_auto_ml_job_v2` API를 호출하여 Autopilot 작업을 시작할 수 있습니다. https://docs.aws.amazon.com/cli/latest/reference/sagemaker/create-auto-ml-job-v2.html

In [ ]:
# cell 07
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-banking-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

sm.create_auto_ml_job_v2(
    AutoMLJobName=auto_ml_job_name,
    AutoMLJobInputDataConfig=auto_ml_job_input_data_config,
    OutputDataConfig=output_data_config,
    AutoMLJobObjective=auto_ml_job_objective,
    AutoMLProblemTypeConfig=auto_ml_problem_type_config,
    RoleArn=role
)

## SageMaker Autopilot 작업 진행 상황 추적<a name="Tracking"></a>
SageMaker Autopilot 작업은 다음과 같은 상위 수준 단계로 구성됩니다:
* 데이터 분석(Analyzing Data): 데이터셋을 분석하고 Autopilot이 데이터셋에서 시도해야 할 ML 파이프라인 목록을 생성합니다. 또한 데이터셋은 훈련 세트와 검증 세트로 분할됩니다.
* 특성 엔지니어링(Feature Engineering): Autopilot이 데이터셋의 개별 특성과 집계 수준에서 특성 변환을 수행합니다.
* 모델 튜닝(Model Tuning): 최고 성능의 파이프라인과 훈련 알고리즘(파이프라인의 마지막 단계)에 대한 최적의 하이퍼파라미터가 선택됩니다.

In [ ]:
# cell 08
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)

## 결과

이제 describe_auto_ml_job_v2 API를 사용하여 SageMaker Autopilot 작업에서 선택한 최고의 후보를 조회해보겠습니다.

In [ ]:
# cell 09
best_candidate = sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

### 최고 후보를 사용하여 배치 추론 수행하기

이제 데이터셋에 대한 SageMaker Autopilot 작업을 성공적으로 완료했으니, [추론 파이프라인](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html)을 사용하여 후보 중 하나로 모델을 생성하겠습니다.

In [ ]:
# cell 10
model_name = 'automl-banking-model-' + timestamp_suffix

model = sm.create_model(
    Containers=best_candidate['InferenceContainers'],
    ModelName=model_name,
    ExecutionRoleArn=role
)

print('Model ARN corresponding to the best candidate is : {}'.format(model['ModelArn']))

Amazon SageMaker 배치 변환을 사용하여 배치 추론을 수행할 수 있습니다. 동일한 모델은 Amazon SageMaker 호스팅을 사용하여 온라인 추론을 수행하도록 배포할 수도 있습니다.

In [ ]:
# cell 11
transform_job_name = 'automl-banking-transform-' + timestamp_suffix

transform_input = {
    'DataSource': {
        'S3DataSource': {
            'S3DataType': 'S3Prefix',
            'S3Uri': test_data_s3_path
        }
    },
    'ContentType': 'text/csv',
    'CompressionType': 'None',
    'SplitType': 'Line'
}

transform_output = {
    'S3OutputPath': 's3://{}/{}/inference-results'.format(bucket,prefix),
}

transform_resources = {
    'InstanceType': 'ml.m5.4xlarge',
    'InstanceCount': 1
}

sm.create_transform_job(
    TransformJobName = transform_job_name,
    ModelName = model_name,
    TransformInput = transform_input,
    TransformOutput = transform_output,
    TransformResources = transform_resources
)

변환 작업이 완료되는지 지켜보겠습니다.

In [ ]:
# cell 12
print ('JobStatus')
print('----------')


describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
job_run_status = describe_response['TransformJobStatus']
print (job_run_status)

while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
    job_run_status = describe_response['TransformJobStatus']
    print (job_run_status)
    sleep(30)

이제 변환 작업의 결과를 살펴보겠습니다:

In [ ]:
# cell 13
s3_output_key = '{}/inference-results/test_data.csv.out'.format(prefix);
local_inference_results_path = 'inference_results.csv'

s3 = boto3.resource('s3')
inference_results_bucket = s3.Bucket(session.default_bucket())

inference_results_bucket.download_file(s3_output_key, local_inference_results_path);

data = pd.read_csv(local_inference_results_path, sep=';')
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

### SageMaker Autopilot이 탐색한 다른 후보 모델 보기
SageMaker Autopilot이 탐색한 모든 후보 모델(다양한 하이퍼파라미터 조합으로 평가된 파이프라인)을 확인하고 최종 성능 지표에 따라 정렬할 수 있습니다.

In [ ]:
# cell 14
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, SortBy='FinalObjectiveMetricValue')['Candidates']
index = 1
for candidate in candidates:
  print (str(index) + "  " + candidate['CandidateName'] + "  " + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))
  index += 1

### 후보 생성 노트북
    
Sagemaker AutoPilot은 후보 정의 노트북도 자동으로 생성합니다. 이 노트북은 Sagemaker Autopilot이 최적의 후보를 찾기 위해 수행한 다양한 단계를 대화식으로 살펴보는 데 사용할 수 있습니다. 또한 이 노트북을 사용하여 병렬 처리, 사용된 하드웨어, 탐색된 알고리즘, 특성 추출 스크립트 등과 같은 다양한 런타임 매개변수를 재정의할 수 있습니다.
    
이 노트북은 다음 Amazon S3 위치에서 다운로드할 수 있습니다:

In [ ]:
# cell 15
sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['CandidateDefinitionNotebookLocation']


### 데이터 탐색 노트북
Sagemaker Autopilot은 데이터 탐색 노트북도 자동으로 생성하며, 다음 Amazon S3 위치에서 다운로드할 수 있습니다:

In [ ]:
# cell 16
sm.describe_auto_ml_job_v2(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['DataExplorationNotebookLocation']


## 정리

Autopilot 작업은 데이터셋 분할, 전처리 스크립트 또는 전처리된 데이터 등과 같은 많은 기본 아티팩트를 생성합니다. 이 코드는 주석 처리를 해제하면 이러한 아티팩트를 삭제합니다. 이 작업은 생성된 모든 모델과 자동 생성된 노트북도 함께 삭제합니다.

In [ ]:
# cell 17
#s3 = boto3.resource('s3')
#bucket = s3.Bucket(bucket)

#job_outputs_prefix = '{}/output/{}'.format(prefix,auto_ml_job_name)
#bucket.objects.filter(Prefix=job_outputs_prefix).delete()